In [1]:
import sys

%load_ext autoreload
%autoreload 2

sys.path.append("..")

from src.make_predictions import *
from src.data.process_data import *

import sqlite3
import joblib
import pandas as pd

In [2]:
final_stacked_model = joblib.load('../models/finalized_stacked_model')

prev_season_elo = pd.read_csv("../data/clean/final_elo_ratings_2020.csv")

In [3]:
connection = sqlite3.connect('../data/nba.db')

df = load_current_season_team_data(connection, 2021)



In [4]:
pd.options.display.max_columns=100
df.loc[54012:54013]

,SEASON,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,E_NET_RATING,NET_RATING,AST_PCT,AST_TOV,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,E_TM_TOV_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE,PCT_FGA_2PT,PCT_FGA_3PT,PCT_PTS_2PT,PCT_PTS_2PT_MR,PCT_PTS_3PT,PCT_PTS_FB,PCT_PTS_FT,PCT_PTS_OFF_TOV,PCT_PTS_PAINT,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,PCT_AST_FGM,PCT_UAST_FGM
54012,2021-22,1610612750,MIN,Minnesota Timberwolves,0022100169,2021-11-10,MIN @ GSW,L,240,39,95,0.411,17,44,0.386,15,20,0.75,17,23,40,30,8,4,17,28,110,-13,106.0,110.0,124.7,121.8,-18.8,-11.8,0.769,1.76,19.8,0.400,0.511,0.449,16.378,17.0,0.495,0.525,1.0,0.200,101.2,100.5,83.75,100.0,0.393,0.542,0.458,0.400,0.018,0.464,0.127,0.136,0.164,0.382,0.636,0.364,0.941,0.059,0.769,0.231
54013,2021-22,1610612744,GSW,Golden State Warriors,0022100169,2021-11-10,GSW vs. MIN,W,240,40,85,0.471,11,40,0.275,32,40,0.80,20,33,53,23,14,6,16,21,123,13,124.7,121.8,106.0,110.0,18.8,11.8,0.575,1.44,16.2,0.489,0.600,0.551,16.227,15.8,0.535,0.599,1.0,0.204,101.2,100.5,83.75,101.0,0.607,0.529,0.471,0.472,0.081,0.268,0.195,0.260,0.146,0.390,0.483,0.517,0.818,0.182,0.575,0.425


In [5]:
home_team = 'PHX'
away_team = 'BOS'
home_spread = -5
away_spread = -home_spread
home_ml = 1.51
away_ml = 2.7

conn = sqlite3.connect('../data/nba.db')
season = 2021

df = load_current_season_team_data(conn, season)

df = clean_team_data(df)

df = prep_for_aggregation(df)

spreads, moneylines = load_current_season_betting_data(conn, 2021)
clean_moneylines = clean_moneyline_df(df = moneylines)
clean_spreads = clean_spreads_df(df = spreads)

full_df = merge_betting_and_boxscore_data(clean_spreads, clean_moneylines, df)
full_df = create_matchups(full_df)
full_df = get_team_and_opp_ewm(full_df, min_periods=1, alpha=0.1)

full_df = add_percentage_features(full_df)
full_df = add_rest_days_for_model(full_df)

current_elo, full_df = get_current_season_elo_ratings(full_df, prev_season_elo)

matchup = create_matchup_rows_for_model(full_df, home_team, away_team,
                                                home_spread, away_spread, home_ml, away_ml)

matchup

..\src\data\process_data.py:145: RuntimeWarning: All-NaN axis encountered
  abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:151: RuntimeWarning: All-NaN axis encountered
  abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:212: RuntimeWarning: All-NaN axis encountered
  lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:217: RuntimeWarning: All-NaN axis encountered
  lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]
progress:: 864it [00:00, 1866.15it/s]


,RECORD_team_diff,FG2M_team_diff,FG2A_team_diff,FG3M_team_diff,FG3A_team_diff,FTM_team_diff,FTA_team_diff,OREB_team_diff,DREB_team_diff,REB_team_diff,AST_team_diff,STL_team_diff,BLK_team_diff,TOV_team_diff,PF_team_diff,PTS_team_diff,PLUS_MINUS_team_diff,E_OFF_RATING_team_diff,E_DEF_RATING_team_diff,E_NET_RATING_team_diff,POSS_team_diff,PIE_team_diff,PTS_2PT_MR_team_diff,PTS_FB_team_diff,PTS_OFF_TOV_team_diff,PTS_PAINT_team_diff,AST_2PM_team_diff,AST_3PM_team_diff,UAST_2PM_team_diff,UAST_3PM_team_diff,ATS_DIFF_team_diff,RECORD_opp_diff,FG2M_opp_diff,FG2A_opp_diff,FG3M_opp_diff,FG3A_opp_diff,FTM_opp_diff,FTA_opp_diff,OREB_opp_diff,DREB_opp_diff,REB_opp_diff,AST_opp_diff,STL_opp_diff,BLK_opp_diff,TOV_opp_diff,PF_opp_diff,PTS_opp_diff,PLUS_MINUS_opp_diff,E_OFF_RATING_opp_diff,E_DEF_RATING_opp_diff,E_NET_RATING_opp_diff,POSS_opp_diff,PIE_opp_diff,PTS_2PT_MR_opp_diff,PTS_FB_opp_diff,PTS_OFF_TOV_opp_diff,PTS_PAINT_opp_diff,AST_2PM_opp_diff,AST_3PM_opp_diff,UAST_2PM_opp_diff,UAST_3PM_opp_diff,ATS_DIFF_opp_diff,TEAM_COVERED_opp_diff,WL_PCT_diff,ATS_PCT_diff,AVG_SPREAD_diff,AVG_ML_diff,AVG_ATS_DIFF_diff,REST_diff,OREB_PCT_team_diff,OREB_PCT_opp_diff,DREB_PCT_team_diff,DREB_PCT_opp_diff,REB_PCT_team_diff,REB_PCT_opp_diff,TS_PCT_team_diff,TS_PCT_opp_diff,EFG_PCT_team_diff,EFG_PCT_opp_diff,AST_RATIO_team_diff,AST_RATIO_opp_diff,TOV_PCT_team_diff,TOV_PCT_opp_diff,team_elo_pred,team_rating_i,opp_rating_i,elo_MOV_pred,SPREAD_team,ML_team
0,0.324659,3.471085,7.370913,-1.204229,-5.188447,-2.699012,-2.386807,-0.460766,0.897231,0.436466,2.453312,1.199211,-1.452355,-0.083187,1.376753,0.630469,4.301568,-0.856269,-5.533504,4.65568,1.495982,0.030442,3.27515,1.06147,1.495557,3.365397,2.510917,-0.446697,0.831049,-0.907475,0.343265,-0.324659,0.881381,4.044977,-1.398363,-3.034878,-1.238773,0.851217,1.10257,0.059109,1.161679,0.357341,0.479581,-0.406921,1.850048,-0.381295,-3.671099,-4.301568,-5.533504,-0.856269,-4.65568,1.505696,-0.030442,-0.487781,-1.068225,-2.432095,2.269912,0.316448,-0.046128,0.342201,-1.445067,-0.343265,-0.043708,0.314815,-0.021164,-4.742165,-0.409721,0.816239,1.0,-0.008269,0.01388,-0.01388,0.008269,-0.00395,0.00395,-0.007083,-0.026965,0.005903,-0.019487,2.095874,0.01794,-0.190902,1.272315,0.805026,1708.738907,1531.406093,8.8,-5.0,1.51
1,-0.324659,-3.471085,-7.370913,1.204229,5.188447,2.699012,2.386807,0.460766,-0.897231,-0.436466,-2.453312,-1.199211,1.452355,0.083187,-1.376753,-0.630469,-4.301568,0.856269,5.533504,-4.65568,-1.495982,-0.030442,-3.27515,-1.06147,-1.495557,-3.365397,-2.510917,0.446697,-0.831049,0.907475,-0.343265,0.324659,-0.881381,-4.044977,1.398363,3.034878,1.238773,-0.851217,-1.10257,-0.059109,-1.161679,-0.357341,-0.479581,0.406921,-1.850048,0.381295,3.671099,4.301568,5.533504,0.856269,4.65568,-1.505696,0.030442,0.487781,1.068225,2.432095,-2.269912,-0.316448,0.046128,-0.342201,1.445067,0.343265,0.043708,-0.314815,0.021164,4.742165,0.409721,-0.816239,-1.0,0.008269,-0.01388,0.01388,-0.008269,0.00395,-0.00395,0.007083,0.026965,-0.005903,0.019487,-2.095874,-0.01794,0.190902,-1.272315,0.194974,1531.406093,1708.738907,-8.8,5.0,2.70


In [9]:
connection = sqlite3.connect('../data/nba.db')


main(connection, 2021, final_stacked_model, prev_season_elo, append=True)

updating basic team boxscores
updating advanced team boxscores
num_games_updated: 0
All team advanced boxscores up to date in season 2021-22
updating scoring boxscores
num_games_updated: 0
All team advanced boxscores up to date in season 2021-22
updating moneyline data
moneyline table is up to date
updating spreads data
Updating spreads for 0 days
spreads table is up to date


..\src\data\process_data.py:145: RuntimeWarning: All-NaN axis encountered
  abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:151: RuntimeWarning: All-NaN axis encountered
  abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:212: RuntimeWarning: All-NaN axis encountered
  lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:217: RuntimeWarning: All-NaN axis encountered
  lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.00it/s]
progress:: 888it [00:00, 4799.83it/s]


,game_date,home_team,away_team,home_spread,home_cover_probs,away_cover_probs,home_cover_avg
0,2021-12-20,BOS,PHI,-3.5,0.525576,0.473612,0.525982
1,2021-12-20,CHI,HOU,-6.5,0.489978,0.511637,0.489170
2,2021-12-20,MEM,OKC,-9.0,0.501245,0.497863,0.501691
3,2021-12-20,UTA,CHA,-12.0,0.493682,0.505836,0.493923
4,2021-12-20,GSW,SAC,-12.5,0.518391,0.480628,0.518882
5,2021-12-20,LAC,SAS,-4.5,0.485505,0.514762,0.485371


In [11]:
final_stacked_model_sfs = joblib.load('../models/finalized_stacked_model_retrain-21-12-20')

connection = sqlite3.connect('../data/nba.db')


main(connection, 2021, final_stacked_model_sfs, prev_season_elo, append=False)


updating basic team boxscores
updating advanced team boxscores
num_games_updated: 0
All team advanced boxscores up to date in season 2021-22
updating scoring boxscores
num_games_updated: 0
All team advanced boxscores up to date in season 2021-22
updating moneyline data
moneyline table is up to date
updating spreads data
Updating spreads for 0 days
spreads table is up to date


..\src\data\process_data.py:145: RuntimeWarning: All-NaN axis encountered
  abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:151: RuntimeWarning: All-NaN axis encountered
  abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:212: RuntimeWarning: All-NaN axis encountered
  lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:217: RuntimeWarning: All-NaN axis encountered
  lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.08it/s]
progress:: 888it [00:00, 4673.81it/s]


,game_date,home_team,away_team,home_spread,home_cover_probs,away_cover_probs,home_cover_avg
0,2021-12-20,BOS,PHI,-3.5,0.544044,0.462055,0.540995
1,2021-12-20,CHI,HOU,-6.5,0.471258,0.537117,0.467070
2,2021-12-20,MEM,OKC,-9.0,0.476305,0.530416,0.472945
3,2021-12-20,UTA,CHA,-12.0,0.504824,0.507187,0.498818
4,2021-12-20,GSW,SAC,-12.5,0.513310,0.502293,0.505508
5,2021-12-20,LAC,SAS,-4.5,0.499987,0.505363,0.497312


In [8]:
final_stacked_model_v2 = joblib.load('../models/final_stacked_clf_v2-21-12-20')

connection = sqlite3.connect('../data/nba.db')

main(connection, 2021, final_stacked_model_v2, prev_season_elo, append=False)


progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:10<00:00, 10.99s/it]
..\src\data\process_data.py:145: RuntimeWarning: All-NaN axis encountered
  abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:151: RuntimeWarning: All-NaN axis encountered
  abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:212: RuntimeWarning: All-NaN axis encountered
  lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:217: RuntimeWarning: All-NaN axis encountered
  lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.21s/it]
progress:: 888it [00:00, 3402.41it/s]


,game_date,home_team,away_team,home_spread,home_cover_probs,away_cover_probs,home_cover_avg
0,2021-12-20,BOS,PHI,-3.5,0.521861,0.475821,0.523020
1,2021-12-20,CHI,HOU,-6.5,0.496895,0.511033,0.492931
2,2021-12-20,MEM,OKC,-9.0,0.490163,0.507536,0.491314
3,2021-12-20,UTA,CHA,-12.0,0.484808,0.515726,0.484541
4,2021-12-20,GSW,SAC,-12.5,0.506350,0.495105,0.505623
5,2021-12-20,LAC,SAS,-4.5,0.484627,0.517539,0.483544
